In [2]:
import os
import astropy.io.fits as fits
import datetime as dt
import gzip
import re
import glob
from pprint import pprint as pp
import csv
import requests
import json
import dateutil as du
import dateutil.parser as dateparser 

import logging


In [13]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [43]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [44]:
root_dir = "/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/M35/"
ALLOWED_EXT = ['.gz', '.bz2']
forbidden_keys = ['HISTORY', 'COMMENT']
DATE_REGEX = '^\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])$'

In [45]:
START_DATETIME = dateparser.parse('1019-01-27T12:06:21')
END_DATETIME = dateparser.parse('2222-01-27T12:06:21')

In [46]:
filters_file = '/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/filters_dict.csv'
objects_file = '/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/objects_dict.csv'
observers_file = '/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/observers_dict.csv'

In [88]:
def check_in_dict(_dict, _name):
    _name = _name.strip()
    _name_up = _name.upper()
    if _name in _dict.keys():
        return _name
    elif _name_up in _dict.keys():
        return _name_up
    else:
        for header, body in _dict.items():
            if _name_up in body: return header
    return _name


def read_names_dict(_file):
    _dict = {}
    with open(_file) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            row = [x for x in row if x]
            if not row: continue
            header = row[0]
            body = set([x.upper() for x in row[1:]])
            _dict[header] = body
    return _dict

def get_dirs_to_walk(data_dir, datetime_start, datetime_end):
    folders_in_root = os.listdir(data_dir)

    date_start = datetime_start.date()
    if datetime_start.hour >= 12:
        date_start = date_start + dt.timedelta(days=1)
    
    date_end = datetime_end.date()
    if datetime_end.hour >=12:
        date_end = date_end + dt.timedelta(days=1)

    p = re.compile(DATE_REGEX) 
    
    dirs_to_walk = []
    for _d in os.listdir(data_dir):
        if (p.match(_d) and \
            date_end >= dt.datetime.strptime(_d, '%Y-%m-%d').date() >= date_start
           ):
            dirs_to_walk.append(os.path.join(data_dir, _d))
    return dirs_to_walk

def get_files(_dir):
    
    files_in_folder = []
    for r, d, f in os.walk(_dir):
        for file in f:
            files_in_folder.append(os.path.join(r, file))
    
    files_to_open = []
    for _file in files_in_folder:
        if os.path.isfile(_file):
            if os.path.splitext(_file)[-1] in ALLOWED_EXT:
                files_to_open.append(_file)
    
    return files_to_open


def get_folder_data(files_to_open, objects_dict, filters_dict, observers_dict):
    folder_data = []
    logger.debug(files_to_open)
    for f in files_to_open:
        try:
            f_in = gzip.open(f, 'rb')
        except OSError as e:
            logger.error(e)
            continue
            # with gzip.open(f, 'rb') as f_in:
        else:
            with f_in:        
                try:
                    hdr = dict(fits.getheader(f, ignore_missing_end=True))
                except OSError as e:
                    logger.warning(f'HDR problem in file: {f} - {e}')
                    continue

                obs_datetime = dt.datetime.strptime(
                    hdr['DATE-OBS'] + 'T' + hdr['TIME-OBS'],
                    '%Y-%m-%dT%H:%M:%S.%f').isoformat()
                
                object_name = check_in_dict(objects_dict, hdr['OBJECT'])
                observers = [
                    check_in_dict(observers_dict, observer) for observer in hdr['OBSERVER'].strip().split()
                ]
                observers = [
                    {'name': observer} for observer in observers
                ]
                color_filter = check_in_dict(filters_dict, hdr['FILTER'])
                exptime = hdr['EXPTIME']
                
            row = {
                'obs_datetime': obs_datetime,
                'object_name': object_name,
                'observers': observers,
                'color_filter': color_filter,
                'exptime': exptime,
            }
            logger.debug(f'data row: {row}')
            folder_data.append(row)
        
    folder_data = sorted(folder_data, key=lambda x: x['obs_datetime'])
    
    return folder_data


def get_grouped_folder_data(folder_data, telescope_name):
    results = {}
    target = {}
    n_frames = 0
    prev_name = None

    for frame in folder_data:
        object_name = str(frame['object_name']).strip()
        if object_name not in results:
            object_dict = {
                'name': object_name,
                'datetime_start': frame['obs_datetime'],
                'observers': frame['observers'],
                'colorfilters': [],
                'total_exposure_time': 0,
                'number_of_frames': 0,
                'telescope': telescope_name,
            }
            results[object_name] = object_dict
        else:
            object_dict = results[object_name]
        object_dict['colorfilters'].append(frame['color_filter'])
        object_dict['datetime_end'] = frame['obs_datetime']
        object_dict['number_of_frames'] += 1
        object_dict['total_exposure_time'] += float(frame.get('exptime', 0))

    for object_name in results.keys():
        object_dict = results[object_name]
        object_dict['total_exposure_time'] = round(
                    object_dict['total_exposure_time'], 2)
        object_dict['colorfilters'] = [
                    {'name': c} for c in set(object_dict['colorfilters'])
                ]

    return results


def send_data(data_to_send):
    for folder_results in data_to_send:
        for _, target_data in folder_results.items():
            try:
                logger.debug(f'Sending data: {target_data}')
                response = requests.post(
                    UPLOAD_URL, auth=(DB_USER, DB_PASSWORD),
                        data=json.dumps(target_data),
                        headers={'content-type': 'application/json'})
            except (requests.ConnectionError, requests.Timeout) as e:
                logger.error(f'No DB connection - {e}')
                raise Exception(f'No connection to DB! - {e}')
            
            if not response.ok:
                logger.error(f'{response.content}\n {target_data}')
    
def send_data(data_to_send):
    for folder_results in data_to_send:
        for _, target_data in folder_results.items():
            try:
                logger.debug(f'Sending data: {target_data}')
                response = requests.post(
                    url='http://127.0.0.1:8000/targets/',
                    auth=(
                        os.getenv('DJANGO_USER'),
                        os.getenv('DJANGO_PASSWORD'),
                        data=json.dumps(target_data),
                        headers={'content-type': 'application/json'})
            except (requests.ConnectionError, requests.Timeout) as e:
                logger.error(f'No DB connection - {e}')
                raise Exception(f'No connection to DB! - {e}')
            
            if not response.ok:
                logger.error(f'{response.content}\n {target_data}')

In [83]:
dirs_to_walk = sorted(get_dirs_to_walk(root_dir, START_DATETIME, END_DATETIME))
data_to_send = []

objects_dict = read_names_dict(objects_file)
filters_dict = read_names_dict(filters_file)
observers_dict = read_names_dict(observers_file)

for _dir in dirs_to_walk:
    print('#'*9)
    pp(_dir)
    folder_files = get_files(_dir)
    folder_data = get_folder_data(folder_files, objects_dict, filters_dict, observers_dict)
    grouped_folder_data = get_grouped_folder_data(folder_data, 'Z60')
    data_to_send.append(grouped_folder_data)
    pp(grouped_folder_data)
    print('#'*9)

#########
'/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/M35/2017-04-02'
{'J1007': {'colorfilters': [{'name': 'R'}],
           'datetime_end': '2017-04-02T23:53:47.150000',
           'datetime_start': '2017-04-02T23:44:32.760000',
           'name': 'J1007',
           'number_of_frames': 10,
           'observers': [{'name': 'MS'}],
           'telescope': 'Z60',
           'total_exposure_time': 600.0},
 'J1427': {'colorfilters': [{'name': 'R'}],
           'datetime_end': '2017-04-03T00:11:18.280000',
           'datetime_start': '2017-04-03T00:02:03.770000',
           'name': 'J1427',
           'number_of_frames': 10,
           'observers': [{'name': 'MS'}],
           'telescope': 'Z60',
           'total_exposure_time': 600.0},
 'J1504': {'colorfilters': [{'name': 'R'}],
           'datetime_end': '2017-04-03T00:28:42.880000',
           'datetime_start': '2017-04-03T00:17:09.170000',
           'name': 'J1504',
           'number_of_frames': 12,
           '